In [ ]:
# -*- coding: utf-8 -*-


import numpy as np
import nidaqmx
import time
import pyvisa as visa
import matplotlib.pyplot as plt
import pandas as pd

#FIRST PART: MODULATE AMPLITUDE
rm = visa.ResourceManager()
gen = rm.open_resource(r'USB0::0x1AB1::0x0642::DG1ZA241701602::INSTR') # Rigol DG1022 noise generator

gen.write('*RST') # Reset the generator
# gen.write('SOUR1:APPL:NOIS 1')  # Set the noise frequency and amplitude
gen.write('SOUR1:APPL:SINE 10000, 0.2') 
gen.write('OUTPut1:STATe On')
# Modulate the input current with external noise
#for i in range(12):
#    
#    #voltage = float(osc.query('MEAS:VAV?'))
#    print(f'Amplitude: {amplitude}')

# Close the instruments

df = pd.DataFrame()
voltage_mean = []
#SECOND PART: READ the VOLTAGE
with nidaqmx.Task() as send_voltage:
    send_voltage.ai_channels.add_ai_voltage_chan("Dev1/ai1") # Set the channel to read from
    send_voltage.timing.cfg_samp_clk_timing(rate=1000) # Set the sampling rate

    # Read the voltage
    for i in range(1, 2):
        amplitude =  3 # Change the amplitude of the noise signal
        # gen.write(f'SOUR1:APPL:SIN {frequ}, 2') # Set the noise amplitude
        gen.write(f'SOUR1:APPL:NOIS {amplitude}')
        time.sleep(1) # Wait for the noise to settle
        voltage = send_voltage.read(number_of_samples_per_channel=5000)
        df[f'{amplitude}V'] = voltage
        voltage_mean.append(np.mean(voltage))
        # print(f'Amplitude: {amplitude}')
        # print(f'Voltage: {voltage_mean:.3f} V')
        
gen.write('OUTPut1:STATe OFF') # Turn off the generator
gen.close()

for i in range(1):
    plt.plot(df.iloc[:, i])
plt.legend()
plt.xlabel('samples')
plt.ylabel('Amplitude (V)')
    
    